In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from anomaly_classifier import AnomalyClassifier

# MLE Mode: 

In [19]:
mode = "MLE"
threshold = "2sigma"

# MSE Mode:

In [18]:
mode = "MSE"
threshold = "max_minus_{}".format("60percent")

## Parameters phm data

In [28]:
param = {
    "data" : {
        "path" : '../../data/phm_data_challenge/recipe/dataset_for_each_recipe/test/test_recipe_67_labeled_short.csv',
        "droped_feature" : ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION", "ROTATIONSPEED", "status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 12,
    },
        "anomaly_detection" : {
        "threshold_quadratic_difference" : 0.5,
        "threshold_q_d_each_sensor" : [3.561851, 3.561851, 3.561851, 3.561851, 
                                      3.561851, 3.561851, 3.561851, 3.561851, 
                                       3.561851, 3.561851, 3.561851, 3.561851],
        "no_standard_deviation" : 2
    },
    "prediction_path": "../visualisation/files/prediction/{}/phm_67_SQ36.csv".format(mode),
    "classification_path" : "../visualisation/files/classification/{}/phm_67_SQ36.csv".format(mode, threshold)
}

## Parameters artifical data

In [39]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/artifical_2_signals_errors.csv',
        "droped_feature" : ["anomaly"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 2,
    },
        "anomaly_detection" : {
        "threshold_quadratic_difference" : 0.5,
        "threshold_q_d_each_sensor" : [0.7012361884117126, 3.1871493458747864], # [0.13679444326055645, 0.5441877437706729],
        "no_standard_deviation" : 2
    },
    "prediction_path": "../visualisation/files/prediction/{}/artfic.csv".format(mode),
    "classification_path" : "../visualisation/files/classification/{}/artfic_{}.csv".format(mode, threshold)
}

## Parameters cpps data

In [14]:
param = {
    "data" : {
        "path" : '../../data/cpps_degradation_new/data_obs10/test/obs_space_error_sinusiod_preprocessed.csv',
        "droped_feature" : ["status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 10,
    },
        "anomaly_detection" : {
        "threshold_quadratic_difference" : 0.5,
        "threshold_q_d_each_sensor" : [0.24372421389777016, 0.24761591192911026, 0.32903928384871034, 
                                       0.18159441767379658, 0.27133063146032477, 0.23155811781422755, 
                                       0.149805263381253, 0.31187742832928145, 0.0716386888894709, 0.2080102174959789],
        "no_standard_deviation" : 1
    },
    "prediction_path": "../visualisation/files/prediction/{}/cpps.csv".format(mode),
    "classification_path" : "../visualisation/files/classification/{}/cpps_{}.csv".format(mode, threshold)
}

## Parameters ramming data

In [20]:
param = {
    "data" : {
        "path" : '../../data/ramming_data/data_train_preprocessed.csv',
        "droped_feature" : ["status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 50,
    },
        "anomaly_detection" : {
        "threshold_quadratic_difference" : 0.5,
        "threshold_q_d_each_sensor" : [],
        "no_standard_deviation" : 2
    },
    "prediction_path": "../visualisation/files/prediction/{}/ramming_train.csv".format(mode),
    "classification_path" : "../visualisation/files/classification/{}/ramming_train_{}.csv".format(mode, threshold)
}

## Detect Anomaly
### Euclidean Distance Metric for the Hole Machine (for MSE Loss Function)

In [34]:
results_prediction = pd.read_csv(param["prediction_path"], sep=";")
anomaly_classifier = AnomalyClassifier()
results = anomaly_classifier.apply_euclidean_distance_metric(results_prediction, 
                                                           param['model']['input_size'], 
                                                           param["anomaly_detection"]["threshold_quadratic_difference"])

### Euclidean Distance Difference Metric for Each Sensor (for MSE Loss Function)

In [64]:
results_prediction = pd.read_csv(param["prediction_path"], sep=";")
anomaly_classifier = AnomalyClassifier()
results = anomaly_classifier.apply_euclidean_distance_metric(results_prediction, 
                                                           param['model']['input_size'],
                                                           param["anomaly_detection"]["threshold_quadratic_difference"],
                                                           percentage = 0.4,
                                                           each_sensor=True, 
                                                           threshold_sensors=param["anomaly_detection"]["threshold_q_d_each_sensor"])

### Standard Deviation Metric (for MLE Loss Function)

In [21]:
results_prediction = pd.read_csv(param["prediction_path"], sep=";")
anomaly_classifier = AnomalyClassifier()
results = anomaly_classifier.apply_prediction_interval_metric(results_prediction, 
                                                            param['model']['input_size'],
                                                            param["anomaly_detection"]["no_standard_deviation"])

In [22]:
results.head()

,ID,obs_1 target,obs_2 target,obs_3 target,obs_4 target,obs_5 target,obs_6 target,obs_7 target,obs_8 target,obs_9 target,...,Anomaly Sensor_41,Anomaly Sensor_42,Anomaly Sensor_43,Anomaly Sensor_44,Anomaly Sensor_45,Anomaly Sensor_46,Anomaly Sensor_47,Anomaly Sensor_48,Anomaly Sensor_49,Anomaly Sensor_50
0,9.0,-0.579954,-1.082348,1.710099,1.208595,-1.094804,-1.857220,0.728437,-1.303208,-0.123162,...,0,0,0,0,0,0,0,0,0,0
1,10.0,-0.581526,-1.102224,1.699215,1.181358,-1.111566,-1.847425,0.740729,-1.339664,-0.181549,...,0,0,0,0,0,0,0,0,0,0
2,11.0,-0.589088,-1.125409,1.730382,1.195109,-1.082208,-1.873643,0.757127,-1.365868,-0.209144,...,0,0,0,0,0,0,0,0,0,0
3,12.0,-0.603368,-1.251806,1.807651,1.177841,-1.019877,-1.935482,0.845683,-1.525055,-0.418739,...,0,0,0,0,0,0,0,0,0,0
4,13.0,-0.599112,-1.077018,1.729705,1.214666,-1.045327,-1.885822,0.716193,-1.289662,-0.112665,...,0,0,0,0,0,0,0,0,0,0


## Combine prediction data with data which was not consider for inference

In [23]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [24]:
complete_data.to_csv(param["classification_path"], sep=";", index=False)